In [1]:
from dotenv import dotenv_values
import praw
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# Download the VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\seb\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Load environment variables
env = dotenv_values('./.env')

In [4]:
# Set up Reddit API credentials
reddit = praw.Reddit(
    client_id=env['R_CLIENT_ID'],
    client_secret=env['R_CLIENT_SECRET'],
    user_agent=env['R_USER_AGENT']
)

In [5]:
env['DB_SERVER']

'127.0.0.1'

In [6]:
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

praw.models.Submission: 
https://praw.readthedocs.io/en/v7.7.1/code_overview/models/submission.html

In [7]:
from time import sleep
import database

In [8]:
# Stream post submissions in selected subreddit and analyze sentiment
subreddit_name = 'Economics'

subreddit = reddit.subreddit(subreddit_name)
submission_data = {}  # submission data storage

for submission in subreddit.stream.submissions():
    if submission.stickied:  # if post is pinned, skip it
        continue
    if not submission.selftext:  # if post doesn't contain text, skip it
        continue
    
    sentiment = sia.polarity_scores(submission.selftext)

    submission_data[f"{submission.id}"] = {\
        'id': submission.id,
        'created_utc': int(submission.created_utc),
        'num_comments': submission.num_comments,
        'score': submission.score,
        'upvote_ratio': submission.upvote_ratio,
        'sentiment': sentiment['compound']
    }
    
    database.insert_row('sentiment', submission_data)

    print(list(submission_data.items())[-1])  ## DEBUG: print last submission collected

OperationalError: connection to server at "127.0.0.1", port 5432 failed: FATAL:  database "reddit_4" does not exist
